<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Train_Fear_RNN_attent_April19%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Despair: Refactored training run - April 12,2023

Synchronize this notebook with the new dataset creation capacity. 

In [1]:
#Need pip install tensorflow with...Jupyter Notebook

!pip install nlp
#!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 1.5 MB/s eta 0:00:00


In [2]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

In [3]:
#Dec 7, 2022: Not sure what this piece of code is used for... 

import warnings
warnings.filterwarnings('ignore')

In [4]:
#Setting column options

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 100)

#Supressing the scientific notation 

pd.set_option('display.float_format', lambda x: '%.3f' % x)

##Importing dataset and fear_axis definition 

In [5]:
#Defining the despair axis
#Removed: April 12, 2023: "content"

#Defining the fear axis 

fear_axis= {
    "dread" : ['panic','dread','horror','horrified','horrifying','terror','terrified','terrifying'],
    "fear":  ["fear", "fearful", "fright","frightening","afraid","frightful","frightfully","frightened",'scared','scary','scare'],
    "anxiety": ["anxious","anxiety",'angst','anxiousness'],
    "worry":["worry", "worried","worrying","worries"],
    "concern": ["concern", "concerning","concerned","concerns"],
    "calm": ["calm","peaceful","serene","serenity","untroubled", "content", "contented", "composed", "tranquil"]
    }

    

In [6]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Load the dataset 


#path = "/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_G2000_266K_April14,2023.csv"


path = "/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_G2000_1.4mil_April19,2023.csv"
df=pd.read_csv(path)

In [8]:
df.shape

(1428572, 6)

In [38]:
df.head(100)

,text,emotion1,label,word_length,treatment,new_word_length,word_count
0,seeming to wait for me and now everything in t...,calm,calm,77,Nothing,77,77
1,would never behave as he does to another guard...,worry,worry,80,Nothing,80,80
2,section there is also the seamen pay branch wh...,concern,concern,68,Nothing,68,68
3,seemed to be well oriented and conscious of ev...,contented,calm,68,Nothing,68,68
4,two sides he stuffed cold bannock into the poc...,panic,dread,72,Nothing,72,72
...,...,...,...,...,...,...,...
95,by the time we arrived in oakland i was as lim...,anxious,anxiety,76,Nothing,76,76
96,details some strong words shall express what i...,fright,fear,69,Nothing,69,69
97,which they make to furnish their huge carcasse...,fright,fear,70,Nothing,70,70
98,to whose grief must be added without any fault...,worry,worry,84,Nothing,84,84


In [10]:
df.columns

Index(['text', 'emotion1', 'emotion2', 'word_length', 'treatment', 'new_word_length'], dtype='object')

In [11]:
#Relabelling a column... if needed... 

df.rename(columns={"emotion2": "label"},inplace =True)

In [12]:
#Let's see if we have missing values...No missing values... 

df.isna().sum()

text               0
emotion1           0
label              0
word_length        0
treatment          0
new_word_length    0
dtype: int64

#Truncating Word Length 

In [13]:
#Defining function to calculate word length

def count_words(text):
    return len(text.split())

df['word_count'] = df['text'].apply(count_words)

In [14]:
df['word_count'].describe()

count   1428572.000
mean         49.925
std          21.537
min           5.000
25%          33.000
50%          43.000
75%          72.000
max          99.000
Name: word_count, dtype: float64

In [15]:
def truncate_text(text, max_len, keyword):
    if len(text) <= max_len:
        return text
    
    # Find the index of the keyword in the text
    keyword_idx = text.find(keyword)
    
    # If the keyword is not found or is too close to the ends of the text, just truncate from the ends
    if keyword_idx == -1 or keyword_idx < max_len//2 or keyword_idx > len(text) - max_len//2:
        return text[:max_len]
    
    # Truncate from both sides
    start_idx = keyword_idx - max_len//2
    end_idx = keyword_idx + max_len//2
    return text[start_idx:end_idx]


In [16]:
# Apply the function to the 'text' column and create a new column 'truncated_text'
#df['truncated_text'] = df.apply(lambda row: truncate_text(row['text'], max_len=100, keyword='emotion1'), axis=1)

In [17]:
#Redoing the word count....I think this needs to get reapplied... 

#df["word_count_trunc"] = df['truncated_text'].apply(count_words)

KeyError: ignored

In [ ]:
#Checking word length again... 

#df['word_count_trunc'].describe()

##Replacing the emotion words in the dataset 

In [18]:
#Generate all the unique emotion words that then get replaced... 

a = df['emotion1'].unique().tolist()

#The list a is our list of variable responses from the dataset... 
keyword = "emotion word"
words = a
for j in words: 
  df['text'] = df['text'].str.replace(j,keyword)

##Counting vocab size 

In [19]:
#Total vocabulary size prior to truncation 

from tensorflow.keras.preprocessing.text import Tokenizer

# convert the text into a list
text = df['text'].tolist()

# create a Tokenizer object
tokenizer = Tokenizer()

# fit the tokenizer on the text data
tokenizer.fit_on_texts(text)

# calculate the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [20]:
vocab_size

127780

##Splitting into train, validate and test datasets inluding shuffle dataset 

In [21]:
#This outputs 3 different dataframes... originally 0.6 and 0.8

train, validate, test = np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

##Preparing Labels - Needs manual label

In [22]:
#Converting the pandas dataframe into a list of labels... 
#We may consider puting this into a function... 

trainlabel=train['label'].tolist()
vallabel=validate['label'].tolist()
testlabel=test['label'].tolist()

In [23]:
#Creating the classes variable... 

classes = list(fear_axis.keys())


In [24]:
# Map each class to a unique integer
classes_to_index = dict((c, i) for i, c in enumerate(classes))

# Map each integer back to its corresponding class
index_to_classes = dict((v, k) for k, v in classes_to_index.items())

In [25]:
#Creating a lambda function...called "names_to_ids"

names_to_ids = lambda labels: np.array([classes_to_index.get(x) for x in labels])

In [26]:
#Applying the names_to_ids functions to the labels

train_labels = names_to_ids(trainlabel)
val_labels = names_to_ids(vallabel)
test_labels = names_to_ids(testlabel)

#Testing out the labels...
print(train_labels[10])

5


##Input Training Text and Tokenizing Tweets

In [27]:
#Converting the different pandas dataframes into a list of text fields...
#Choice of variables: truncated_text, text_minus1, 'filtered_text',''filtered_text_minus1'

traintext=train['text'].tolist()
valtext=validate['text'].tolist()
testtext=test['text'].tolist()

In [28]:
#Importing the tokenizer...
#Input into the tokenizer is a list

from tensorflow.keras.preprocessing.text import Tokenizer

In [29]:
#Input into the tokenizer is a list

tokenizer = Tokenizer(num_words=128945, oov_token='<UNK>')

#I think that this is the missing piece...I'm not really sure what it does...  

tokenizer.fit_on_texts(traintext)
#tokenizer.fit_on_texts(valtext)

#Testing the tokenization... 

#print(tokenizer.texts_to_sequences([tweets[10]]))

#Padding and Truncating Sequences 

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
#Creating a function that tokenizes and pads the sequences...

def get_sequences(tokenizer, texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=300, padding='post')
    return padded_sequences

In [32]:
#Applying the function to tokenize and pad... to all test,validate and test
#Syntax: val_sequences = get_sequences(tokenizer, val_tweets)

padded_train_sequences = get_sequences(tokenizer, traintext)
val_sequence = get_sequences(tokenizer,valtext)
test_sequence = get_sequences(tokenizer,testtext)


##Creating and training the model 

In [33]:
#Embedding layer works on number of tokens -> approximately words... 
#Adding another layer: tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(500, return_sequences=True)),

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(128945, 400, input_length=300),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(400, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(400)),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 400)          51578000  
                                                                 
 bidirectional (Bidirectiona  (None, 300, 800)         2563200   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 800)              3843200   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 4806      
                                                                 
Total params: 57,989,206
Trainable params: 57,989,206
Non-trainable params: 0
_________________________________________________________________


#Adding an attention layer to the model 

In [34]:
from tensorflow.keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

# Define your model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(128945, 400, input_length=200),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True)),
    Attention(),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 400)          51578000  
                                                                 
 bidirectional_2 (Bidirectio  (None, 200, 400)         961600    
 nal)                                                            
                                                                 
 attention (Attention)       (None, 400)               600       
                                                                 
 dense_1 (Dense)             (None, 6)                 2406      
                                                                 
Total params: 52,542,606
Trainable params: 52,542,606
Non-trainable params: 0
_________________________________________________________________


In [35]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(128945, 400, input_length=300),
    tf.keras.layers.Conv1D(200, 5, activation='relu'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(400, return_sequences=True)),
    Attention(),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 400)          51578000  
                                                                 
 conv1d (Conv1D)             (None, 296, 200)          400200    
                                                                 
 bidirectional_3 (Bidirectio  (None, 296, 800)         1923200   
 nal)                                                            
                                                                 
 attention_1 (Attention)     (None, 800)               1096      
                                                                 
 dense_2 (Dense)             (None, 6)                 4806      
                                                                 
Total params: 53,907,302
Trainable params: 53,907,302
Non-trainable params: 0
__________________________________________

In [36]:
import numpy as np

# Compute class frequencies
class_freqs = np.bincount(train_labels)
total_samples = np.sum(class_freqs)

# Compute class weights
class_weights = {}
for i, freq in enumerate(class_freqs):
    class_weights[i] = total_samples / (len(class_freqs) * freq)



In [37]:
#Most up to date architecture: April 15, 2023 -> Fa

#Having the callbacks option on means that the model stops once you are like over-generalizing... 
#Let's leave the callback option on for now...


h = model.fit(
    padded_train_sequences, train_labels,
    validation_data=(val_sequence, val_labels),
    epochs=3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
   ]
)

Epoch 1/3
19134/35715 [===============>..............] - ETA: 23:35 - loss: 0.8148 - accuracy: 0.6862

KeyboardInterrupt: ignored

In [ ]:
# Train the model with class weights
#h = model.fit(
    padded_train_sequences, train_labels,
    validation_data=(val_sequence, val_labels),
    epochs=2,
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
   ]
)


#Evaluating The Model 

In [ ]:
def show_history(h):
    epochs_trained = len(h.history['loss'])
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(0, epochs_trained), h.history.get('accuracy'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_accuracy'), label='Validation')
    plt.ylim([0., 1.])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(0, epochs_trained), h.history.get('loss'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_loss'), label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

show_history(h)

In [ ]:
#Running the model on the test sequence and test labels... 

eval = model.evaluate(test_sequence, test_labels)

In [ ]:
#preds = model.predict_classes(test_sequence)
preds=model.predict(test_sequence) 
classes_x=np.argmax(preds,axis=1)
preds.shape, test_labels.shape

##Error Analysis 

In [ ]:
#Dumb luck metric 

counts = df['label'].value_counts()
dumb_luck = max(counts) / sum(counts)

dumb_luck


In [ ]:
#Creating an inference dataframe 

inf_df=pd.DataFrame({
    'data':testtext,
    "labels_predicted": classes_x                    
})
inf_df["labels_predicted_marked"]=inf_df['labels_predicted'].apply(lambda x: index_to_classes[x])
inf_df["actual_labels"]=testlabel

#Creating the labels index datastructure...

inf_df["actual_label_index"]=inf_df['actual_labels'].apply(lambda x: classes_to_index[x])



In [ ]:
#Checking the inf_df shape...

inf_df.shape

In [ ]:
#Making the correct prediction 

inf_df['correct_pred']=0 # first assigning all to 0.
inf_df.loc[(inf_df['labels_predicted']==inf_df['actual_label_index']),'correct_pred']=1 # labelling 1 if the prediction is right.

In [ ]:
# magnitutde of error
inf_df['error_magnitude']=abs(inf_df['labels_predicted']-inf_df['actual_label_index'])

In [ ]:
# Count the frequency of each class in the error_magnitude column
counts = inf_df['error_magnitude'].value_counts()

# Create a histogram with one bar for each class
plt.bar(counts.index, counts.values)

# Set the title and axis labels
plt.suptitle('Histogram of Error Magnitude in Fear Classification')
plt.xlabel('Magnitude')
plt.ylabel('Frequency')

##Saving the model and evaluation dataset

In [ ]:
#So this works but we need to make sure that we install Keras as a dependency 

from keras.models import load_model

#model.save('/content/drive/MyDrive/Sean/Emoclass_Dec2022/Model_Fear_151K_Feb23,2022')

In [ ]:
#Exporting the evaluation dataset 

In [ ]:
#Exporting the dataset 

#path = "/content/drive/MyDrive/Sean/Emoclass/Train_Fear_Error_Analysis_151K_March8,2023.csv"
inf_df.to_csv(path)
